In [47]:
def blight_Model():
    
    # import the necessary libraries

    import pandas as pd
    import numpy as np
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.linear_model import LinearRegression

    # Load the Dataset

    train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    test = pd.read_csv('test.csv')
    address = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    # Cleaning preprocessing the data, if you find it hard, please do refer to the pratice file under the same file name to
    # to understand the rationale behind it (since i have tired to show the data after each step of cleaning)
    
    # Dealing with only US Data
        
    train = train[train.country == 'USA'] # Dealing with only US Data
    test = test[test.country == "USA"]
    
    # Merging address data with lat and long data
    
    add_with_latlon = pd.merge(address, latlons, on = 'address') 
    
    # Merging the new address and lat long data with train and test
    
    train = pd.merge(train, add_with_latlon, on = 'ticket_id') 
    test = pd.merge(test, add_with_latlon, on = 'ticket_id')
    
    # Drop all the column with considerable amount of NA values (>50%)
    # it could be found using the code (uncomment the next line)
    # train.isna().mean()*100
    
    train = train.drop(['violation_zip_code', 'non_us_str_code', 'payment_date','collection_status', 'grafitti_status'], axis = 1)
    
    # Remove all the unnecessary columns
    
    train = train.drop(['agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country','payment_amount', 'balance_due', 
        'payment_status', 'compliance_detail','hearing_date', 'ticket_issued_date','violation_description', 'address'], axis = 1)
    
    test = test.drop(['agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_description', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status', 'address', 'lat',
       'lon'], axis =1)

    
    # using disposition and violation codes has the two features
    
    string_features = ['disposition', 'violation_code']
    train =  pd.get_dummies(train,columns = string_features, drop_first= True)
    test =  pd.get_dummies(test,columns = string_features, drop_first= True)  
    
    
    # Remove all Na from other columns
    
    train = train.dropna()
    
    
    # Creating X and y, one with compliance and the other without compliance
    
    X = train.drop('compliance', axis =1)
    y = train['compliance']
    
    # remove all features from train which is not in test data file but is found in 'X'

    train_features = X
    train_features_set = set(train_features)
    
    for feature in set(train_features):
        if feature not in test:
            train_features_set.remove(feature)
    train_features = list(train_features_set)

    X = X[train_features]
    test = test[train_features]

    
    # Use random forest regressor to predict, as it performed well compared to other methods (Check the practice file)
    
    reg = RandomForestRegressor(max_depth = 6,random_state=0).fit(X, y)
    prob = reg.predict(test)    
    
    return pd.DataFrame(prob, test.ticket_id)    

In [48]:
blight_Model() 

C:\Users\HP\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,0
ticket_id,
284932,0.123528
285362,0.047908
285361,0.058664
285338,0.047908
285346,0.207990
...,...
376496,0.091530
376497,0.091530
376499,0.240857


In [34]:
pd.DataFrame(prob, test.ticket_id)

,0
ticket_id,
284932,0.123528
285362,0.047908
285361,0.058664
285338,0.047908
285346,0.207990
...,...
376496,0.091530
376497,0.091530
376499,0.240857


In [30]:
X.shape

(159867, 135)

In [16]:
X.shape

(159867, 252)

In [32]:
y.shape

(159867,)

In [31]:
test.shape

(61001, 135)

In [28]:
prob.shape

(61001,)